In [1]:
import os
import numpy as np
import re
import pandas as pd
import json
import boto3
from scipy import stats
import datetime as dt 
from pytz import timezone
import pytz
import sqlalchemy
import plotly.offline as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Read in Data Sheet
- Add two variables that will help with grouping and summarization

In [2]:
os.chdir('/home/ec2-user/SageMaker/Andrew/General/Upskill/inputs')
upskill_df = pd.read_excel('Spreadsheet Challenge.xlsx',             
                           sheet_name='Data') 
upskill_df['Expiring Month'] = upskill_df.Expiration.astype(str).str[:7]
upskill_df['Earthquake/Flood Risk'] =  upskill_df.Earthquake.astype(str) + ', ' + upskill_df.Flood.astype(str)
upskill_df.head()

,Policy,Expiration,Location,State,Region,Insured Value,Construction Type,Business Type,Earthquake,Flood,Expiring Month,Earthquake/Flood Risk
0,100242,2021-01-02,Urban,NY,East,1617630,Frame,Retail,N,N,2021-01,"N, N"
1,100314,2021-01-02,Urban,NY,East,8678500,Fire Resist,Apartment,Y,Y,2021-01,"Y, Y"
2,100359,2021-01-02,Rural,WI,Midwest,2052660,Frame,Farming,N,N,2021-01,"N, N"
3,100315,2021-01-03,Urban,NY,East,17580000,Frame,Apartment,Y,Y,2021-01,"Y, Y"
4,100385,2021-01-03,Urban,NY,East,1925000,Masonry,Hospitality,N,N,2021-01,"N, N"


In [3]:
print("Total Insured: ${:,.0f}".format(upskill_df['Insured Value'].sum()))
len(np.unique(upskill_df.State))

Total Insured: $2,482,205,481


10

# Descriptive Summaries grouped by variables of interest from Question 2
- State
- Flooding zone
- Location
- Business type
- Construction type 


In [38]:
fig = px.violin(upskill_df, x='Insured Value',y='Region',box=True,
                hover_data=upskill_df.columns,
                color='Region',
                title='Distribution of Insured Value by Region',width=600)#height=500
               
fig.update_layout(legend=dict(orientation="h",                            
                             yanchor="bottom",
                             y=1,                             
                             xanchor="right",                             
                             x=1                                
                            )
                 )
fig.show()

In [32]:
def describe_groupby(df,group_vars):
    print(group_vars)
    display(df.groupby(group_vars)['Insured Value'].describe())
    return

pivot_cats = ['Region','State','Location','Construction Type', 'Business Type', 'Earthquake', 'Flood']
for i in pivot_cats:
    describe_groupby(upskill_df, [i])
    fig = px.box(upskill_df, x='Insured Value',y=upskill_df[i],#box=True,
                hover_data=upskill_df.columns,
                color=upskill_df[i],
                title='Distribution of Insured Value by {}'.format(str(i))#height=500
               )
    fig.update_layout(legend=dict(orientation="h",                            
                             yanchor="bottom",
                             y=-.3,                             
                             xanchor="right",                             
                             x=.5                                
                            )
                 )
    fig.show()
describe_groupby(upskill_df,['Region','State'])

['Region']


,count,mean,std,min,25%,50%,75%,max
Region,,,,,,,,
Central,21.0,2.346362e+06,2.740931e+06,218490.0,608500.00,1569440.0,1991600.0,10617800.0
East,337.0,5.418462e+06,6.267541e+06,30000.0,1761960.00,2815000.0,7611000.0,49837500.0
Midwest,108.0,4.340066e+06,6.877028e+06,105000.0,1636098.75,2393327.5,4341515.0,53410614.0
Northeast,34.0,4.064205e+06,3.989282e+06,97920.0,1356763.75,2488600.0,6127275.5,16429900.0


['State']


,count,mean,std,min,25%,50%,75%,max
State,,,,,,,,
IL,14.0,6.634327e+06,8.011814e+06,172100.0,1966812.5,3514500.0,7092668.75,29128000.0
ME,4.0,2.484139e+06,3.133354e+06,552300.0,730575.0,1117127.5,2870691.25,7150000.0
MI,7.0,2.555560e+06,3.712828e+06,315000.0,497610.0,714000.0,2623450.00,10617800.0
MN,2.0,1.569650e+06,3.741302e+05,1305100.0,1437375.0,1569650.0,1701925.00,1834200.0
NH,7.0,7.660286e+05,8.707310e+05,97920.0,232840.0,345000.0,976800.00,2500000.0
NJ,76.0,4.391707e+06,5.372994e+06,205000.0,1667425.0,2813779.5,4670431.50,36356000.0
NY,261.0,5.717441e+06,6.483750e+06,30000.0,1780000.0,2815900.0,8678500.00,49837500.0
OH,14.0,2.241762e+06,2.271631e+06,218490.0,796947.5,1654270.0,1833725.00,8272853.0
VT,23.0,5.342792e+06,4.086782e+06,1325200.0,2315837.5,3579800.0,6778860.00,16429900.0


['Location']


...

,count,mean,std,min,25%,50%,75%,max
Earthquake,,,,,,,,
N,244.0,2.409452e+06,1.973122e+06,100000.0,1298761.25,1995950.0,2938850.0,10979275.0
Y,256.0,7.399606e+06,7.710628e+06,30000.0,2277466.25,4792629.0,10123850.0,53410614.0


['Flood']


,count,mean,std,min,25%,50%,75%,max
Flood,,,,,,,,
N,251.0,2.405614e+06,1.904281e+06,100000.0,1296622.5,2013357.0,3007400.0,10979275.0
Y,249.0,7.543760e+06,7.780011e+06,30000.0,2269865.0,5056900.0,10302000.0,53410614.0


['Region', 'State']


count          mean           std        min        25%  \
Region    State                                                            
Central   MI       7.0  2.555560e+06  3.712828e+06   315000.0   497610.0   
          OH      14.0  2.241762e+06  2.271631e+06   218490.0   796947.5   
East      NJ      76.0  4.391707e+06  5.372994e+06   205000.0  1667425.0   
          NY     261.0  5.717441e+06  6.483750e+06    30000.0  1780000.0   
Midwest   IL      14.0  6.634327e+06  8.011814e+06   172100.0  1966812.5   
          MN       2.0  1.569650e+06  3.741302e+05  1305100.0  1437375.0   
          WI      92.0  4.051166e+06  6.736066e+06   105000.0  1555850.0   
Northeast ME       4.0  2.484139e+06  3.133354e+06   552300.0   730575.0   
          NH       7.0  7.660286e+05  8.707310e+05    97920.0   232840.0   
          VT      23.0  5.342792e+06  4.086782e+06  1325200.0  2315837.5   

                       50%         75%         max  
Region    State                                     
Central   MI      714000.0  2623450.00  10617800.0  
          OH     1654270.0  1833725.00   8272853.0  
East      NJ     2813779.5  4670431.50  36356000.0  
          NY     2815900.0  8678500.00  49837500.0  
Midwest   IL     3514500.0  7092668.75  29128000.0  
          MN     1569650.0  1701925.00   1834200.0  
          WI     2300950.0  4149900.00  53410614.0  
Northeast ME     1117127.5  2870691.25   7150000.0  
          NH      345000.0   976800.00   2500000.0  
          VT     3579800.0  6778860.00  16429900.0

# Build State Level Dataframes, Explore State Level Stats
- No risk: policies with no risk of flood or earthquake

In [5]:
upskill_by_state_df = upskill_df.groupby('State')['Insured Value'].sum().reset_index()
upskill_by_state_no_risk_df = upskill_df[(upskill_df.Flood == 'N') &
                                         (upskill_df.Earthquake == 'N')].groupby('State')['Insured Value'].sum().reset_index()


In [22]:
upskill_state_joined_df = pd.merge(upskill_by_state_df,
                                   upskill_by_state_no_risk_df.rename(columns={'Insured Value':'No Flood/Earthquake Insured Value'}),
                                   on='State',
                                   how='left')
upskill_state_joined_df.fillna(value=0,inplace=True)
upskill_state_joined_df['% of Insured Value at Flood/Earthquake Risk'] = \
                       1- upskill_state_joined_df['No Flood/Earthquake Insured Value']/upskill_state_joined_df['Insured Value']
upskill_state_joined_df['Prop Insured Value'] = upskill_state_joined_df['Insured Value']/upskill_df['Insured Value'].sum()
upskill_state_joined_df

,State,Insured Value,No Flood/Earthquake Insured Value,% of Insured Value at Flood/Earthquake Risk,Prop Insured Value
0,IL,92880575,23233675.0,0.749854,0.037419
1,ME,9936555,2786555.0,0.719565,0.004003
2,MI,17888920,7271120.0,0.593541,0.007207
3,MN,3139300,0.0,1.000000,0.001265
4,NH,5362200,5264280.0,0.018261,0.002160
5,NJ,333769700,23084169.0,0.930838,0.134465
6,NY,1492252118,248158670.0,0.833702,0.601180
7,OH,31384673,26007673.0,0.171326,0.012644
8,VT,122884213,41996632.0,0.658242,0.049506
9,WI,372707227,158665728.0,0.574289,0.150152


In [23]:
(upskill_state_joined_df['Insured Value'].sum() \
 - upskill_state_joined_df['No Flood/Earthquake Insured Value'].sum())\
/ upskill_state_joined_df['Insured Value'].sum()




0.783874257749252

In [8]:
fig = px.bar(upskill_state_joined_df, x='State', y='Insured Value', color='State',text='Insured Value',)
fig.update_yaxes(tickformat='$5,.0')

fig.update_traces(texttemplate='$%{text:,.0}', textposition='auto')

fig.update_layout(title='Insured Value by State')
fig.show()

# Explore Total Insured Policy Value by Natural Risks

In [9]:
fig = px.bar(upskill_df.groupby(['Earthquake/Flood Risk'])['Insured Value'].sum().reset_index(), 
                 x='Earthquake/Flood Risk', 
                 y='Insured Value',
                 text='Insured Value',
                 color='Earthquake/Flood Risk',
            )
         
fig.update_traces(texttemplate='$%{text:5,.0}', textposition='auto',)
fig.update_yaxes(tickformat='$5,.0')

fig.update_layout(title='Total Insured by Flood/Earthquake Risk',
                 #legend_orientation='h'
                 )
fig.show()

In [28]:
fig = px.bar(upskill_df.groupby(['Earthquake/Flood Risk','Region'])['Insured Value'].sum().reset_index(), 
                 x='Region', 
                 y='Insured Value',
                 text='Insured Value',
                 color='Earthquake/Flood Risk',
            )
         
fig.update_traces(texttemplate='$%{text:5,.0}', textposition='auto',)
fig.update_yaxes(tickformat='$5,.0')

fig.update_layout(title='Total Insured by Flood/Earthquake Risk',
                 legend=dict(orientation="h",                            
                             yanchor="bottom",
                             y=1,                             
                             xanchor="right",                             
                             x=1                                
                            )
                 
                 )
fig.show()

In [10]:
fig = px.bar(upskill_df.groupby(['Earthquake/Flood Risk','Expiring Month'])['Insured Value'].sum().reset_index(), 
                 x='Expiring Month', 
                 y='Insured Value',
                 text='Insured Value',
                 color='Earthquake/Flood Risk',
            )
         
fig.update_traces(texttemplate='$%{text:5,.0}', textposition='auto',)
fig.update_yaxes(tickformat='$5,.0')

fig.update_layout(title='Monthly Expiring Policy Value by Flood/Earthquake Risk',
                 #legend_orientation='h'
                 )
fig.show()

In [31]:
fig = px.bar(upskill_df[(upskill_df.Earthquake == 'N') & (upskill_df.Flood == 'N') 
                           ].groupby(['Region',
                                      'Expiring Month'])['Insured Value'].sum().reset_index().sort_values('Region'), 
                 x='Expiring Month', 
                 y='Insured Value',
                 text='Region',
                 color='Region',
            )
         
#fig.update_traces(texttemplate='$%{text:5,.0}', textposition='auto',)
fig.update_yaxes(tickformat='$5,.0')

fig.update_layout(title='Monthly Expiring Policy Value for Policies WITHOUT Flood or Earthquake Risk by Region',
                 legend=dict(orientation="h",                            
                             yanchor="bottom",
                             y=1,                             
                             xanchor="right",                             
                             x=1                                
                            ))
fig.show()

In [12]:
fig = px.bar(upskill_df[(upskill_df.Earthquake == 'Y') | 
                            (upskill_df.Flood == 'Y') 
                           ].groupby(['State',
                                      'Expiring Month'])['Insured Value'].sum().reset_index().sort_values('State'), 
                 x='Expiring Month', 
                 y='Insured Value',
                 text='State',
                 color='State',
            )
         
#fig.update_traces(texttemplate='$%{text:5,.0}', textposition='auto',)
fig.update_yaxes(tickformat='$5,.0')

fig.update_layout(title='Monthly Expiring Policy Value for Policies WITH Flood or Earthquake Risk by State')
fig.show()

In [13]:
fig = px.bar(upskill_state_joined_df, 
             x='State', color='% of Insured Value at Flood/Earthquake Risk',
             y='Insured Value',
             text='Insured Value',

             #text='% of Insured Value at Flood/Earthquake Risk',
             labels={'% of Insured Value at Flood/Earthquake Risk':'% of Insured Value<br>At Flood/Earthquake Risk'},
            color_continuous_scale='RdYlGn_r',)
         
fig.update_yaxes(tickformat='$5,.0',title='Total Insured Value')

fig.update_traces(texttemplate='$%{text:5,.0} Total', textposition='auto',)

fig.update_layout(title='Insured Value by State - Shading: % of Insured Value At Flood/Earthquake Risk',legend_orientation='h')
fig.show()

In [14]:
fig = px.bar(upskill_by_state_no_risk_df, x='State', y='Insured Value', color='State',text='Insured Value',)
fig.update_yaxes(tickformat='$5,.0')

fig.update_traces(texttemplate='$%{text:,.0}', textposition='auto')

fig.update_layout(title='Total Insured Value without Earthquake or Flooding Risk by State')
fig.show()